In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
plt.style.use('ggplot')
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv ("C:\\Users\\nilli\\Downloads\\house-prices-advanced-regression-techniques\\train.csv")

In [3]:
df= data.copy()

In [4]:
df['Total_size']= df['BsmtFinSF1']+ df['BsmtFinSF2'] + df['1stFlrSF'] +df['2ndFlrSF']+df['GarageArea']

In [5]:
def helper(x):
    if x.BldgType == "TwnhsE":
        return 'Twnhs'
    else:
        return x.BldgType

df['BldgType_clean']=df.apply(helper,axis=1)

In [6]:
df['Outdoor_Porch_Size']= df['WoodDeckSF']+df['OpenPorchSF']

In [7]:
df["AllPorchSF"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]


In [8]:
def recent_constr(x):
    if(x.YearBuilt < x.YearRemodAdd):
        return x.YearRemodAdd
    else: 
        return x.YearBuilt

#defining intermediate column 2    
df['Recent_yr'] = df.apply(recent_constr,axis=1)

#time to sale helper defined:
def time_to_sale(x):
    return x.YrSold - x.Recent_yr

#new column defined to be used in Train
df['Age_at_sale']=df.apply(time_to_sale,axis=1)

In [9]:
#Removing the outlier
df.drop([523,1298],axis=0,inplace=True)

In [10]:
X2=df[["Neighborhood", "OverallQual", "Total_size", "KitchenQual", "Age_at_sale", "ExterQual",
                   "YrSold",  "LotArea", 'BldgType_clean', 'Outdoor_Porch_Size', "Fireplaces", "SalePrice"]]

In [11]:
X2.dtypes

Neighborhood          object
OverallQual            int64
Total_size             int64
KitchenQual           object
Age_at_sale            int64
ExterQual             object
YrSold                 int64
LotArea                int64
BldgType_clean        object
Outdoor_Porch_Size     int64
Fireplaces             int64
SalePrice              int64
dtype: object

In [12]:
X2["Logprice"] = np.log(X2["SalePrice"])
X2["Logprice"]

C:\Users\nilli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: Logprice, Length: 1458, dtype: float64

In [13]:
#Dummifying Neighborhood, ExterQual, KitchenQual, and BldgType columns

from sklearn.preprocessing import OneHotEncoder

#dummify Neighborhood column
Neighborhood_ohe = OneHotEncoder( drop='first', sparse = False )
Neighborhood_enc = Neighborhood_ohe.fit_transform( X2[['Neighborhood']] )
Neighborhood_enc = pd.DataFrame( Neighborhood_enc, columns=Neighborhood_ohe.get_feature_names(['Neighborhood']) )
X2 = pd.concat( (X2.drop(["Neighborhood"], axis=1).reset_index(drop = True), Neighborhood_enc), axis = 1 )

#dummify ExterQual Column
ExterQual_ohe = OneHotEncoder( drop='first', sparse = False )
ExterQual_enc = ExterQual_ohe.fit_transform( X2[['ExterQual']] )
ExterQual_enc = pd.DataFrame( ExterQual_enc, columns=ExterQual_ohe.get_feature_names(['ExterQual']) )
X2 = pd.concat( (X2.drop(["ExterQual"], axis=1).reset_index(drop = True), ExterQual_enc), axis = 1 )

# dummify Fireplaces column
Fireplaces_ohe = OneHotEncoder( drop='first', sparse = False, categories="auto" )
Fireplaces_enc = Fireplaces_ohe.fit_transform(X2[['Fireplaces']] )
Fireplaces_enc = pd.DataFrame( Fireplaces_enc, columns=Fireplaces_ohe.get_feature_names(['Fireplaces']) )
X2 = pd.concat( (X2.drop(["Fireplaces"], axis=1).reset_index(drop = True), Fireplaces_enc), axis = 1 )


# dummify KitchenQual column
from sklearn.preprocessing import OneHotEncoder
KitchenQual_ohe = OneHotEncoder( drop='first', sparse = False )
KitchenQual_enc = KitchenQual_ohe.fit_transform( X2[['KitchenQual']] )
KitchenQual_enc = pd.DataFrame( KitchenQual_enc, columns=KitchenQual_ohe.get_feature_names(['KitchenQual']) )
X2 = pd.concat( (X2.drop(["KitchenQual"], axis=1).reset_index(drop = True), KitchenQual_enc), axis = 1 )

#dummify Building Type
BldgType_clean_ohe = OneHotEncoder( drop='first', sparse = False )
BldgType_clean_enc = BldgType_clean_ohe.fit_transform( X2[['BldgType_clean']] )
BldgType_clean_enc = pd.DataFrame( BldgType_clean_enc, columns=BldgType_clean_ohe.get_feature_names(['BldgType_clean']) )
X2 = pd.concat( (X2.drop(["BldgType_clean"], axis=1).reset_index(drop = True), BldgType_clean_enc), axis = 1 )


X2.head()

,OverallQual,Total_size,Age_at_sale,YrSold,LotArea,Outdoor_Porch_Size,SalePrice,Logprice,Neighborhood_Blueste,Neighborhood_BrDale,...,ExterQual_TA,Fireplaces_1,Fireplaces_2,Fireplaces_3,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,BldgType_clean_2fmCon,BldgType_clean_Duplex,BldgType_clean_Twnhs
0,7,2964,5,2008,8450,61,208500,12.247694,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,6,2700,31,2007,9600,298,181500,12.109011,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,7,2880,6,2008,11250,42,223500,12.317167,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,7,2575,36,2006,9550,35,140000,11.849398,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8,3689,8,2008,14260,276,250000,12.429216,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
y = X2[["Logprice"]]
X = X2.drop(["SalePrice", "Logprice"], axis=1)
print(X.shape)
print(y.shape)

(1458, 42)
(1458, 1)


In [15]:
np.random.seed(0)
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split( X,y, test_size = 0.2  )
print(Xtrain.shape)
print(ytrain.shape)

(1166, 42)
(1166, 1)


In [16]:
Xtrain.head()

,OverallQual,Total_size,Age_at_sale,YrSold,LotArea,Outdoor_Porch_Size,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,ExterQual_TA,Fireplaces_1,Fireplaces_2,Fireplaces_3,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,BldgType_clean_2fmCon,BldgType_clean_Duplex,BldgType_clean_Twnhs
1170,6,2356,50,2008,9120,0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
75,4,1747,36,2009,1596,221,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
390,5,2383,58,2008,8405,130,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1282,6,1824,39,2010,9400,128,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
682,9,3429,5,2007,11248,302,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xtrain = pd.DataFrame( ss.fit_transform(Xtrain), columns = Xtrain.columns)
Xtrain.head()

,OverallQual,Total_size,Age_at_sale,YrSold,LotArea,Outdoor_Porch_Size,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,ExterQual_TA,Fireplaces_1,Fireplaces_2,Fireplaces_3,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,BldgType_clean_2fmCon,BldgType_clean_Duplex,BldgType_clean_Twnhs
0,-0.062929,-0.116190,1.288237,0.126946,-0.132876,-0.972997,-0.029298,-0.101974,-0.204943,-0.144968,...,0.774242,1.134140,-0.299532,-0.058671,-0.170664,-0.814456,0.981307,-0.144968,-0.183543,-0.333968
1,-1.545262,-0.839015,0.612514,0.874518,-0.840150,0.579381,-0.029298,-0.101974,-0.204943,-0.144968,...,0.774242,-0.881725,-0.299532,-0.058671,-0.170664,-0.814456,0.981307,-0.144968,-0.183543,2.994297
2,-0.804096,-0.084144,1.674364,0.126946,-0.200087,-0.059833,-0.029298,-0.101974,-0.204943,-0.144968,...,0.774242,-0.881725,-0.299532,-0.058671,-0.170664,-0.814456,0.981307,-0.144968,-0.183543,-0.333968
3,-0.062929,-0.747623,0.757311,1.622090,-0.106555,-0.073882,-0.029298,-0.101974,-0.204943,-0.144968,...,0.774242,-0.881725,-0.299532,-0.058671,-0.170664,-0.814456,0.981307,-0.144968,5.448322,-0.333968
4,2.160570,1.157357,-0.883730,-0.620626,0.067162,1.148353,-0.029298,-0.101974,-0.204943,-0.144968,...,-1.291585,1.134140,-0.299532,-0.058671,-0.170664,1.227814,-1.019049,-0.144968,-0.183543,-0.333968


In [18]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [19]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso2 = Lasso(warm_start = True, max_iter = 1e7)

params = {'alpha':np.logspace(-6, 0, 100)}
grid_search_lasso = GridSearchCV( estimator=lasso2, param_grid=params, cv=5 )
grid_search_lasso.fit(Xtrain,ytrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=10000000.0, normalize=False,
                             positive=False, precompute=False,
                             random_state=None, selection='cyclic', tol=0.0001,
                             warm_start=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([1.00000000e-06, 1.14975700e-06, 1.32194115e-06, 1.51991108e-06,
       1.74752840...
       7.05480231e-02, 8.11130831e-02, 9.32603347e-02, 1.07226722e-01,
       1.23284674e-01, 1.41747416e-01, 1.62975083e-01, 1.87381742e-01,
       2.15443469e-01, 2.47707636e-01, 2.84803587e-01, 3.27454916e-01,
       3.76493581e-01, 4.32876128e-01, 4.97702356e-01, 5.72236766e-01,
       6.57933225e-01, 7.56463328e-01, 8.69749003e-01, 1.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, ver

In [20]:
grid_search_lasso.best_params_

{'alpha': 7.56463327554629e-05}

In [21]:
grid_search_lasso.cv_results_

{'mean_fit_time': array([0.01902475, 0.01140623, 0.01000824, 0.01038852, 0.0112308 ,
        0.01063805, 0.00951266, 0.0115653 , 0.01143055, 0.01167407,
        0.00909944, 0.01068435, 0.01132088, 0.01093888, 0.00951509,
        0.00998383, 0.00817509, 0.00840831, 0.00952983, 0.00804005,
        0.00908837, 0.00971766, 0.00634499, 0.00771523, 0.0082262 ,
        0.00641322, 0.00562859, 0.0075973 , 0.00542965, 0.00663171,
        0.00770407, 0.00636315, 0.00454946, 0.00636096, 0.00531354,
        0.00596533, 0.00689521, 0.00365896, 0.00602617, 0.00507498,
        0.00391121, 0.00329762, 0.00316558, 0.00422368, 0.00349145,
        0.00361776, 0.00353742, 0.00336194, 0.00302529, 0.00192161,
        0.0037219 , 0.00275745, 0.00393853, 0.00282993, 0.0018239 ,
        0.00258846, 0.00183764, 0.00274377, 0.00110364, 0.00211692,
        0.00241389, 0.00180998, 0.00302114, 0.00131803, 0.00260153,
        0.00345721, 0.00345688, 0.00079775, 0.00172658, 0.0026545 ,
        0.00271506, 0.0025424 ,

In [22]:
grid_search_lasso.best_score_

0.8775028478027199

In [23]:
lasso2.fit(Xtest, ytest).coef_
lst = []
for i in (list(zip(lasso2.fit(Xtest, ytest).coef_, Xtest.columns))):
    lst.append(i)
lst

[(0.0, 'OverallQual'),
 (0.00034642292202006293, 'Total_size'),
 (-0.0038182603941353307, 'Age_at_sale'),
 (0.0, 'YrSold'),
 (-1.6032732639394225e-06, 'LotArea'),
 (0.0003164225693521263, 'Outdoor_Porch_Size'),
 (-0.0, 'Neighborhood_Blueste'),
 (-0.0, 'Neighborhood_BrDale'),
 (0.0, 'Neighborhood_BrkSide'),
 (-0.0, 'Neighborhood_ClearCr'),
 (0.0, 'Neighborhood_CollgCr'),
 (0.0, 'Neighborhood_Crawfor'),
 (-0.0, 'Neighborhood_Edwards'),
 (0.0, 'Neighborhood_Gilbert'),
 (-0.0, 'Neighborhood_IDOTRR'),
 (-0.0, 'Neighborhood_MeadowV'),
 (-0.0, 'Neighborhood_Mitchel'),
 (-0.0, 'Neighborhood_NAmes'),
 (0.0, 'Neighborhood_NPkVill'),
 (-0.0, 'Neighborhood_NWAmes'),
 (0.0, 'Neighborhood_NoRidge'),
 (0.0, 'Neighborhood_NridgHt'),
 (-0.0, 'Neighborhood_OldTown'),
 (-0.0, 'Neighborhood_SWISU'),
 (-0.0, 'Neighborhood_Sawyer'),
 (-0.0, 'Neighborhood_SawyerW'),
 (0.0, 'Neighborhood_Somerst'),
 (0.0, 'Neighborhood_StoneBr'),
 (-0.0, 'Neighborhood_Timber'),
 (0.0, 'Neighborhood_Veenker'),
 (-0.0, 'ExterQu

In [24]:
from sklearn.metrics import mean_squared_error
print("R^2: {}".format(lasso2.score(Xtest, ytest)))
mse=(mean_squared_error(ytest, lasso2.predict(Xtest)))
print("Mean Squared Error: {}".format(mse))
rmse = np.sqrt(mean_squared_error(ytest, lasso2.predict(Xtest)))
print("Root Mean Squared Error: {}".format(rmse))  

R^2: 0.7796673158726266
Mean Squared Error: 0.03734078498001408
Root Mean Squared Error: 0.19323763862150167


In [25]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso3 = Lasso(warm_start = True, max_iter = 1e7)

params = {'alpha':np.linspace(8.697490026177834e-04, 0.0001, 100)}
grid_search_lasso = GridSearchCV( estimator=lasso3, param_grid=params, cv=5 )
grid_search_lasso.fit(Xtrain,ytrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=10000000.0, normalize=False,
                             positive=False, precompute=False,
                             random_state=None, selection='cyclic', tol=0.0001,
                             warm_start=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.00086975, 0.00086197, 0.0008542 , 0.00084642, 0.00083865,
       0.00083087, 0.000...
       0.00028661, 0.00027883, 0.00027106, 0.00026328, 0.0002555 ,
       0.00024773, 0.00023995, 0.00023218, 0.0002244 , 0.00021663,
       0.00020885, 0.00020108, 0.0001933 , 0.00018553, 0.00017775,
       0.00016998, 0.0001622 , 0.00015443, 0.00014665, 0.00013888,
       0.0001311 , 0.00012333, 0.00011555, 0.00010778, 0.0001    ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
grid_search_lasso.best_params_

{'alpha': 0.0001}

In [27]:
grid_search_lasso.best_score_

0.8774965381008835

In [28]:
lasso3.fit(Xtest, ytest).coef_
lst = []
for i in (list(zip(lasso3.fit(Xtest, ytest).coef_, Xtest.columns))):
    lst.append(i)
lst

[(0.0, 'OverallQual'),
 (0.00034642292202006293, 'Total_size'),
 (-0.0038182603941353307, 'Age_at_sale'),
 (0.0, 'YrSold'),
 (-1.6032732639394225e-06, 'LotArea'),
 (0.0003164225693521263, 'Outdoor_Porch_Size'),
 (-0.0, 'Neighborhood_Blueste'),
 (-0.0, 'Neighborhood_BrDale'),
 (0.0, 'Neighborhood_BrkSide'),
 (-0.0, 'Neighborhood_ClearCr'),
 (0.0, 'Neighborhood_CollgCr'),
 (0.0, 'Neighborhood_Crawfor'),
 (-0.0, 'Neighborhood_Edwards'),
 (0.0, 'Neighborhood_Gilbert'),
 (-0.0, 'Neighborhood_IDOTRR'),
 (-0.0, 'Neighborhood_MeadowV'),
 (-0.0, 'Neighborhood_Mitchel'),
 (-0.0, 'Neighborhood_NAmes'),
 (0.0, 'Neighborhood_NPkVill'),
 (-0.0, 'Neighborhood_NWAmes'),
 (0.0, 'Neighborhood_NoRidge'),
 (0.0, 'Neighborhood_NridgHt'),
 (-0.0, 'Neighborhood_OldTown'),
 (-0.0, 'Neighborhood_SWISU'),
 (-0.0, 'Neighborhood_Sawyer'),
 (-0.0, 'Neighborhood_SawyerW'),
 (0.0, 'Neighborhood_Somerst'),
 (0.0, 'Neighborhood_StoneBr'),
 (-0.0, 'Neighborhood_Timber'),
 (0.0, 'Neighborhood_Veenker'),
 (-0.0, 'ExterQu

In [30]:
from sklearn.metrics import mean_squared_error
print("R^2: {}".format(lasso3.score(Xtest, ytest)))
mse=(mean_squared_error(ytest, lasso3.predict(Xtest)))
print("Mean Squared Error: {}".format(mse))
rmse = np.sqrt(mean_squared_error(ytest, lasso3.predict(Xtest)))
print("Root Mean Squared Error: {}".format(rmse))  

R^2: 0.7796673158726266
Mean Squared Error: 0.03734078498001408
Root Mean Squared Error: 0.19323763862150167
